In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


## 依照指示取出模型特定層的資訊

In [4]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [5]:
outputs = []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [6]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [7]:
outputs

[tensor([[[[1.0659, 0.4759, 0.9153,  ..., 0.5783, 0.5211, 0.3612],
           [0.0581, 0.8136, 1.7281,  ..., 0.9193, 0.6414, 0.6112],
           [0.9130, 0.8853, 0.8602,  ..., 1.3969, 0.8500, 0.2443],
           ...,
           [0.9061, 0.2862, 0.8420,  ..., 0.3284, 0.3332, 0.6502],
           [0.8085, 0.5206, 1.1390,  ..., 0.9586, 0.9569, 0.7294],
           [0.9450, 0.6991, 0.3952,  ..., 0.4297, 0.6807, 0.0140]],
 
          [[0.5913, 0.2353, 0.5243,  ..., 0.6073, 0.0633, 0.2509],
           [0.0000, 0.8257, 0.0000,  ..., 0.5884, 0.6846, 0.7796],
           [0.2668, 0.0000, 0.0000,  ..., 0.9637, 0.4766, 0.3439],
           ...,
           [1.0793, 0.6889, 0.0000,  ..., 0.9524, 0.0860, 1.1231],
           [0.4966, 0.8374, 0.2276,  ..., 0.1797, 0.6733, 0.7849],
           [0.1077, 0.2367, 0.1690,  ..., 0.6323, 0.5011, 0.7073]],
 
          [[0.6378, 0.7622, 0.8074,  ..., 0.3446, 0.1655, 0.6920],
           [0.0000, 0.8825, 0.5400,  ..., 1.0481, 0.3415, 0.3939],
           [0.2671, 0.77

In [9]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [10]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            "自行填入"
            torch.nn.init.ones_(m.bias)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [11]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


